In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv

In [2]:
# Carrega variáveis de ambiente do arquivo .env
load_dotenv()
scopus_api = os.getenv("scopus_api")

In [3]:
# Definindo a URL da API
url = "https://api.elsevier.com/content/search/scopus"

# Parâmetros para a requisição
params = {
    "query": '(TITLE ( "market* segmentation" AND "data-driven" ) OR KEY ( "marketing" AND "data-driven" )) AND DOCTYPE(ar)',
    "apiKey": scopus_api,
    "count": 200,  # Definindo o número máximo de registros por página
    "start": 0  # Começando a partir do primeiro registro
}

all_results = []  # Lista para armazenar todos os resultados

while True:
    # Fazendo a requisição GET
    response = requests.get(url=url, params=params)

    # Verificando se a requisição foi bem sucedida
    if response.status_code == 200:
        # Transformando a resposta em um objeto JSON
        data = response.json()

        # Verificando se a chave 'search-results' está presente na resposta
        entries = data['search-results'].get('entry')

        if entries:
            all_results.extend(entries)
        else:
            print("Página sem novos resultados. Encerrando a busca.")
            break

        # Verificando se há mais resultados para recuperar
        if int(data['search-results']['opensearch:itemsPerPage']) == 0:
            # Se não houver mais resultados, interrompa o loop
            break

        # Atualizando o índice do próximo registro a ser recuperado
        params['start'] += params['count']
    else:
        # Tenta capturar a mensagem de erro detalhada
        try:
            error_json = response.json()
            error_info = error_json.get("service-error", {}).get("status", {})
            status_code = error_info.get("statusCode", "Desconhecido")
            status_text = error_info.get("statusText", "Sem mensagem")

            print(f"Erro da API: {status_code} - {status_text}")
        except Exception as e:
            print("Erro inesperado:", e)

        break

# Imprimindo o número total de resultados obtidos
print("Total de resultados:", len(all_results))

df_results = pd.DataFrame(all_results)


Página sem novos resultados. Encerrando a busca.
Total de resultados: 144


In [4]:
# Removendo colunas desnecessárias
columns = ['@_fa', 
           'link',
           'prism:url',
           'eid',
           'pii',
           'subtype',
           'source-id',
           'openaccess',
           'openaccessFlag',
           'article-number',
           'freetoread',
           'freetoreadLabel',
           'prism:aggregationType',
           'prism:issn',
           'prism:volume',
           'prism:issueIdentifier',
           'prism:pageRange',
           'prism:coverDisplayDate',
           'prism:isbn',
           'prism:eIssn',
           'pubmed-id']


df_results = df_results.drop(columns=columns)

In [5]:
# Tratando a coluna dc:identifier - Removendo o prefixo 'SCOPUS_ID:'
df_results['dc:identifier'] = df_results['dc:identifier'].str.replace('SCOPUS_ID:', '')

# Tratando a coluna affiliation - Transformando em uma lista de strings
# Removendo [] da coluna affiliation
df_results['affiliation'] = df_results['affiliation'].apply(lambda x: [aff['affilname'] for aff in x][0] if isinstance(x, list) and x else None)

# Filtrando para registros que são artigos
df_results = df_results[df_results['subtypeDescription'] == 'Article']

In [6]:
# Adicionando a coluna 'abstract' ao DataFrame

# Função para obter o resumo com base no DOI
def get_abstract(doi):
    url = f"https://api.elsevier.com/content/abstract/doi/{doi}?apiKey={scopus_api}"
    headers = {'Accept': 'application/json'}
    response = requests.get(url, headers=headers)
    data = response.json()
    try:
        abstract_content = data["abstracts-retrieval-response"]["item"]["bibrecord"]["head"]["abstracts"]
        return abstract_content
    except KeyError:
        return None

# Itera sobre os valores da coluna prism:doi e obtém os resumos
df_results['abstract'] = df_results['prism:doi'].apply(get_abstract)

In [7]:
# Exportando dataframe como csv
df_results.to_csv('base_scopus_ddm.csv', index=False)
df_results.to_excel('base_scopus_ddm.xlsx', index=False)

# Configurando llm para analisar a base de dados

In [8]:
# ✅ Instale os pacotes antes (uma vez):
# pip install pandas openai python-dotenv tiktoken
from openai import OpenAI

In [9]:
# Carrega variáveis do .env
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [10]:
# ✅ Carrega a base filtrada
df = pd.read_csv("base_scopus_ddm.csv", usecols=[
    "dc:title", "dc:creator", "prism:publicationName", "prism:coverDate",
    "prism:doi", "citedby-count", "affiliation", "subtypeDescription", "abstract"
])

In [11]:
# ✅ Gera uma versão estruturada de cada registro
df["resumo_estruturado"] = df.apply(lambda row: f"""
Título: {row['dc:title']}
Autores: {row['dc:creator']}
Periódico: {row['prism:publicationName']}
Data: {row['prism:coverDate']}
DOI: {row['prism:doi']}
Citações: {row['citedby-count']}
Afilição: {row['affiliation']}
Tipo: {row['subtypeDescription']}
Resumo: {row['abstract']}
""", axis=1)

In [12]:
# 🔁 Limita para os primeiros 200 artigos (ajustável)
texto_completo = "\n\n".join(df["resumo_estruturado"].head(200).tolist())

In [27]:
# ✅ Histórico do chat
history = [
    {"role": "system", "content": "Você é um analista de dados bibliométricos. Use somente a base fornecida para responder."},
    {"role": "user", "content": f"Aqui está a base: \n\n{texto_completo}"}
]

In [29]:
# ✅ Loop de conversa com base Scopus
while True:
    user_input = input("\nVocê: ")
    if user_input.lower() in ["sair", "exit", "quit"]:
        break

    history.append({"role": "user", "content": user_input})

    response = client.chat.completions.create(
        model="o4-mini-2025-04-16",
        messages=history
    )

    bot_reply = response.choices[0].message.content
    print(f"\nIA: {bot_reply}")
    history.append({"role": "assistant", "content": bot_reply})


IA: A seguir alguns dos estudos da base que exploram algorítmos de clustering “avançados” em segmentação de mercado, indo além do simples K-Means:

1) Budaya I.G.B.A. (2025)  
   “Application of Formal Concept Analysis and Clustering Algorithms to Analyze Customer Segments”  
   Jurnal Resti, DOI:10.29207/resti.v9i2.6184  
   • Usa K-Means e GMM para definir clusters em dados RFM.  
   • Em seguida aplica Formal Concept Analysis (FCA) para categorizar as combinações dos atributos (High/Low R, F, M), enriquecendo a interpretação dos segmentos.

2) Uddin M.A. (2024)  
   “Data-driven strategies for digital native market segmentation using clustering”  
   International Journal of Cognitive Computing in Engineering, DOI:10.1016/j.ijcce.2024.04.002  
   • Compara K-Means, MiniBatch K-Means, AGNES (clustering hierárquico) e Fuzzy C-Means.  
   • Avalia via índices de silhueta a capacidade de cada técnica em agrupar interesses similares de consumidores digitais (adolescentes).

3) Chen J.H.